# Plot the Diurnal Cycle of Prediction Errors for a Given Balancing Authority

In [1]:
# Start by importing the packages we need:
import os
import datetime
import yaml

import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt

from glob import glob
from matplotlib import pyplot 
from mpl_toolkits.axes_grid1 import make_axes_locatable


## Set the Directory Structure

In [2]:
# Identify the data input and image output directories:
statistics_input_directory = '/Users/burl878/Documents/Code/code_repos/burleyson-etal_2025_ldrd/data/'
composite_input_directory = '/Users/burl878/Documents/Code/code_repos/burleyson-etal_2025_ldrd/data/composite_projections/'
ba_to_process_input_directory =  '/Users/burl878/Documents/Code/code_repos/burleyson-etal_2025_ldrd/data/'
image_output_directory =  '/Users/burl878/Documents/Code/code_repos/burleyson-etal_2025_ldrd/figures/'


## Set the List of Balancing Authorities to Analyze

BAs used in this analysis are controlled by a master file `balancing_authorities_modeled.yml` stored in the `/data` directory.

In [3]:
# Read the yml file into a dictionary:
with open((ba_to_process_input_directory + 'balancing_authority_modeled.yml'), 'r') as yml:
     ba_list = yaml.load(yml, Loader=yaml.FullLoader)
     bas = [i for i in ba_list.keys()]

# Return the list of BAs to process/plot:
bas


['AZPS', 'BPAT', 'CISO', 'ERCO', 'FPL', 'ISNE', 'PJM', 'SWPP']

## Process the Diurnal Cycle of Load and Errors for a Given Balancing Authority


In [4]:
def process_ba_diurnal_errors(ba_to_process: str, composite_input_directory: str):
    
    # Load in the compiled projection file:
    ba_df = pd.read_csv((composite_input_directory + ba_to_process + '_Composite_Data.csv'), index_col=None, header=0)

    # Replace missing values with NaN:
    ba_df.replace(-999.0, np.nan, inplace=True)
    
    # Set "Time_UTC" to a datetime variable and extract the year, month, and hour:
    ba_df['Time_UTC'] = pd.to_datetime(ba_df['Time_UTC'])
    ba_df['Year'] = ba_df['Time_UTC'].dt.year
    ba_df['Month'] = ba_df['Time_UTC'].dt.month
    ba_df['Hour'] = ba_df['Time_UTC'].dt.hour 

    # Create a season variable:
    ba_df['Season'] = 'N/A'
    ba_df.loc[ba_df['Month'].isin((1, 2, 12)), 'Season'] = 'DJF'
    ba_df.loc[ba_df['Month'].isin((3, 4, 5)), 'Season'] = 'MAM'
    ba_df.loc[ba_df['Month'].isin((6, 7, 8)), 'Season'] = 'JJA'
    ba_df.loc[ba_df['Month'].isin((9, 10, 11)), 'Season'] = 'SON'
    
    # Calculate the hourly error (absolute) for each MLP model:
    ba_df['MLP1_Error_MWh'] = ba_df['Demand_MWh'] - ba_df['MLP1_MWh']
    ba_df['MLP2_Error_MWh'] = ba_df['Demand_MWh'] - ba_df['MLP2_MWh']
    ba_df['MLP3_Error_MWh'] = ba_df['Demand_MWh'] - ba_df['MLP3_MWh']
    ba_df['MLP4_Error_MWh'] = ba_df['Demand_MWh'] - ba_df['MLP4_MWh']
    ba_df['MLP5_Error_MWh'] = ba_df['Demand_MWh'] - ba_df['MLP5_MWh']
    ba_df['MLP6_Error_MWh'] = ba_df['Demand_MWh'] - ba_df['MLP6_MWh']
    ba_df['MLP7_Error_MWh'] = ba_df['Demand_MWh'] - ba_df['MLP7_MWh']

    # Initiate a counter and empty dataframe to store the results:
    counter = 0;
    output_df = pd.DataFrame()

    # Loop over the years, seasons, and hours and calculate the mean value and mean error:
    for year in range(2018,2025,1):
        for season in ['DJF', 'MAM', 'JJA', 'SON']:
            for hour in range(0,24,1):
                
                # Iterate the counter by one:
                counter = counter + 1

                # Subset the data to the given year, season, hour combination:
                subset_df = ba_df.loc[(ba_df['Year'] == year) & (ba_df['Season'] == season) & (ba_df['Hour'] == hour)]

                # Put the mean values into the output dataframe:
                output_df.loc[counter, 'BA'] = ba_to_process
                output_df.loc[counter, 'Year'] = int(year)
                output_df.loc[counter, 'Season'] = season
                output_df.loc[counter, 'Hour'] = int(hour)
                output_df.loc[counter, 'Demand_MWh'] = subset_df['Demand_MWh'].mean().round(1)
                output_df.loc[counter, 'MLP1_MWh'] = subset_df['MLP1_MWh'].mean(skipna=False).round(1)
                output_df.loc[counter, 'MLP2_MWh'] = subset_df['MLP2_MWh'].mean(skipna=False).round(1)
                output_df.loc[counter, 'MLP3_MWh'] = subset_df['MLP3_MWh'].mean(skipna=False).round(1)
                output_df.loc[counter, 'MLP4_MWh'] = subset_df['MLP4_MWh'].mean(skipna=False).round(1)
                output_df.loc[counter, 'MLP5_MWh'] = subset_df['MLP5_MWh'].mean(skipna=False).round(1)
                output_df.loc[counter, 'MLP6_MWh'] = subset_df['MLP6_MWh'].mean(skipna=False).round(1)
                output_df.loc[counter, 'MLP7_MWh'] = subset_df['MLP7_MWh'].mean(skipna=False).round(1)
                output_df.loc[counter, 'MLP1_Error_MWh'] = subset_df['MLP1_Error_MWh'].mean(skipna=False).round(1)
                output_df.loc[counter, 'MLP2_Error_MWh'] = subset_df['MLP2_Error_MWh'].mean(skipna=False).round(1)
                output_df.loc[counter, 'MLP3_Error_MWh'] = subset_df['MLP3_Error_MWh'].mean(skipna=False).round(1)
                output_df.loc[counter, 'MLP4_Error_MWh'] = subset_df['MLP4_Error_MWh'].mean(skipna=False).round(1)
                output_df.loc[counter, 'MLP5_Error_MWh'] = subset_df['MLP5_Error_MWh'].mean(skipna=False).round(1)
                output_df.loc[counter, 'MLP6_Error_MWh'] = subset_df['MLP6_Error_MWh'].mean(skipna=False).round(1)
                output_df.loc[counter, 'MLP7_Error_MWh'] = subset_df['MLP7_Error_MWh'].mean(skipna=False).round(1)

                # Clean up and move to the next step in the loop:
                del subset_df
            
    return output_df


In [5]:
# Test the function:
output_df = process_ba_diurnal_errors(ba_to_process = 'PJM',
                                      composite_input_directory = composite_input_directory)

output_df


,BA,Year,Season,Hour,Demand_MWh,MLP1_MWh,MLP2_MWh,MLP3_MWh,MLP4_MWh,MLP5_MWh,MLP6_MWh,MLP7_MWh,MLP1_Error_MWh,MLP2_Error_MWh,MLP3_Error_MWh,MLP4_Error_MWh,MLP5_Error_MWh,MLP6_Error_MWh,MLP7_Error_MWh
1,PJM,2018.0,DJF,0.0,105372.4,104256.7,NaN,NaN,NaN,NaN,NaN,NaN,1115.8,NaN,NaN,NaN,NaN,NaN,NaN
2,PJM,2018.0,DJF,1.0,104898.8,101900.8,NaN,NaN,NaN,NaN,NaN,NaN,2998.1,NaN,NaN,NaN,NaN,NaN,NaN
3,PJM,2018.0,DJF,2.0,103542.9,99285.3,NaN,NaN,NaN,NaN,NaN,NaN,4257.6,NaN,NaN,NaN,NaN,NaN,NaN
4,PJM,2018.0,DJF,3.0,100710.7,96447.8,NaN,NaN,NaN,NaN,NaN,NaN,4263.0,NaN,NaN,NaN,NaN,NaN,NaN
5,PJM,2018.0,DJF,4.0,96582.3,93540.9,NaN,NaN,NaN,NaN,NaN,NaN,3041.3,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668,PJM,2024.0,SON,19.0,89547.5,92877.0,93142.6,93064.8,90889.6,92163.6,90652.7,92234.0,-3329.4,-3595.0,-3517.3,-1342.1,-2616.1,-1105.2,-2686.4
669,PJM,2024.0,SON,20.0,90506.0,92925.3,93498.9,93322.1,91060.5,92765.1,91063.3,92860.1,-2419.3,-2992.9,-2816.2,-554.5,-2259.1,-557.3,-2354.1
670,PJM,2024.0,SON,21.0,91675.6,92595.1,93619.4,93331.6,90844.0,93112.5,91258.3,93019.9,-919.5,-1943.8,-1656.0,831.6,-1436.9,417.3,-1344.3
671,PJM,2024.0,SON,22.0,93010.2,92087.0,93442.1,93151.9,90389.9,93008.6,91173.4,92919.7,923.2,-431.8,-141.7,2620.3,1.7,1836.8,90.5


## Create a Function to Create the Diurnal Cycle Plot for a Given Balancing Authority


In [32]:
def plot_ba_diurnal_errors(ba_to_plot: str, season_to_plot: str, composite_input_directory: str, image_output_directory: str, image_resolution: int, save_images=False):

    # Process the diurnal error statistics using the function defined above:
    df = process_ba_diurnal_errors(ba_to_process = ba_to_plot,
                                   composite_input_directory = composite_input_directory)

    # Calculate the y-limits for the plot:
    season_df = df.loc[(df['Season'] == season_to_plot)]
    y_min = 0.95*season_df[['Demand_MWh','MLP1_MWh','MLP2_MWh','MLP3_MWh','MLP4_MWh','MLP5_MWh','MLP6_MWh','MLP7_MWh']].min(axis=1).min()
    y_max = 1.05*season_df[['Demand_MWh','MLP1_MWh','MLP2_MWh','MLP3_MWh','MLP4_MWh','MLP5_MWh','MLP6_MWh','MLP7_MWh']].max(axis=1).max()
    
    # Subset the data to individual years:
    y2018_df = df.loc[(df['Year'] == 2018) & (df['Season'] == season_to_plot)]
    y2019_df = df.loc[(df['Year'] == 2019) & (df['Season'] == season_to_plot)]
    y2020_df = df.loc[(df['Year'] == 2020) & (df['Season'] == season_to_plot)]
    y2021_df = df.loc[(df['Year'] == 2021) & (df['Season'] == season_to_plot)]
    y2022_df = df.loc[(df['Year'] == 2022) & (df['Season'] == season_to_plot)]
    y2023_df = df.loc[(df['Year'] == 2023) & (df['Season'] == season_to_plot)]
    y2024_df = df.loc[(df['Year'] == 2024) & (df['Season'] == season_to_plot)]
    
    # Make the plot:
    plt.figure(figsize=(60, 30))
    plt.rcParams['font.size'] = 18
    plt.subplot(241)
    plt.grid(True)
    plt.plot(y2018_df['Hour'], y2018_df['Demand_MWh'], color='black', linestyle='-', label='Obs', linewidth=3)
    plt.plot(y2018_df['Hour'], y2018_df['MLP1_MWh'], color='rosybrown', linestyle='-', label=('MLP1 Corr=' + str(y2018_df['Demand_MWh'].corr(y2018_df['MLP1_MWh']).round(2))), linewidth=3)
    plt.legend(loc='upper left', prop={'size': 18})
    plt.xlim([0, 23])
    plt.ylim([y_min, y_max])
    plt.xlabel('Hour of Day [UTC]', fontsize=18)
    plt.ylabel('Load [MWh]', fontsize=18)
    plt.title(('Load in ' + ba_to_plot + ' 2018: ' + season_to_plot), fontsize=21)
    plt.title('a)', loc='left', fontsize=18)

    plt.subplot(242)
    plt.grid(True)
    plt.plot(y2019_df['Hour'], y2019_df['Demand_MWh'], color='black', linestyle='-', label='Obs', linewidth=3)
    plt.plot(y2019_df['Hour'], y2019_df['MLP1_MWh'], color='rosybrown', linestyle='-', label=('MLP1 Corr=' + str(y2019_df['Demand_MWh'].corr(y2019_df['MLP1_MWh']).round(2))), linewidth=3)
    plt.plot(y2019_df['Hour'], y2019_df['MLP2_MWh'], color='firebrick', linestyle='-', label=('MLP2 Corr=' + str(y2019_df['Demand_MWh'].corr(y2019_df['MLP2_MWh']).round(2))), linewidth=3)
    plt.legend(loc='upper left', prop={'size': 18})
    plt.xlim([0, 23])
    plt.ylim([y_min, y_max])
    plt.xlabel('Hour of Day [UTC]', fontsize=18)
    plt.ylabel('Load [MWh]', fontsize=18)
    plt.title(('Load in ' + ba_to_plot + ' 2019: ' + season_to_plot), fontsize=21)
    plt.title('b)', loc='left', fontsize=18)

    plt.subplot(243)
    plt.grid(True)
    plt.plot(y2020_df['Hour'], y2020_df['Demand_MWh'], color='black', linestyle='-', label='Obs', linewidth=3)
    plt.plot(y2020_df['Hour'], y2020_df['MLP1_MWh'], color='rosybrown', linestyle='-', label=('MLP1 Corr=' + str(y2020_df['Demand_MWh'].corr(y2020_df['MLP1_MWh']).round(2))), linewidth=3)
    plt.plot(y2020_df['Hour'], y2020_df['MLP2_MWh'], color='firebrick', linestyle='-', label=('MLP2 Corr=' + str(y2020_df['Demand_MWh'].corr(y2020_df['MLP2_MWh']).round(2))), linewidth=3)
    plt.plot(y2020_df['Hour'], y2020_df['MLP3_MWh'], color='darkorange', linestyle='-', label=('MLP3 Corr=' + str(y2020_df['Demand_MWh'].corr(y2020_df['MLP3_MWh']).round(2))), linewidth=3)
    plt.legend(loc='upper left', prop={'size': 18})
    plt.xlim([0, 23])
    plt.ylim([y_min, y_max])
    plt.xlabel('Hour of Day [UTC]', fontsize=18)
    plt.ylabel('Load [MWh]', fontsize=18)
    plt.title(('Load in ' + ba_to_plot + ' 2020: ' + season_to_plot), fontsize=21)
    plt.title('c)', loc='left', fontsize=18)

    plt.subplot(244)
    plt.grid(True)
    plt.plot(y2021_df['Hour'], y2021_df['Demand_MWh'], color='black', linestyle='-', label='Obs', linewidth=3)
    plt.plot(y2021_df['Hour'], y2021_df['MLP1_MWh'], color='rosybrown', linestyle='-', label=('MLP1 Corr=' + str(y2021_df['Demand_MWh'].corr(y2021_df['MLP1_MWh']).round(2))), linewidth=3)
    plt.plot(y2021_df['Hour'], y2021_df['MLP2_MWh'], color='firebrick', linestyle='-', label=('MLP2 Corr=' + str(y2021_df['Demand_MWh'].corr(y2021_df['MLP2_MWh']).round(2))), linewidth=3)
    plt.plot(y2021_df['Hour'], y2021_df['MLP3_MWh'], color='darkorange', linestyle='-', label=('MLP3 Corr=' + str(y2021_df['Demand_MWh'].corr(y2021_df['MLP3_MWh']).round(2))), linewidth=3)
    plt.plot(y2021_df['Hour'], y2021_df['MLP4_MWh'], color='olivedrab', linestyle='-', label=('MLP4 Corr=' + str(y2021_df['Demand_MWh'].corr(y2021_df['MLP4_MWh']).round(2))), linewidth=3)
    plt.legend(loc='upper left', prop={'size': 18})
    plt.xlim([0, 23])
    plt.ylim([y_min, y_max])
    plt.xlabel('Hour of Day [UTC]', fontsize=18)
    plt.ylabel('Load [MWh]', fontsize=18)
    plt.title(('Load in ' + ba_to_plot + ' 2021: ' + season_to_plot), fontsize=21)
    plt.title('d)', loc='left', fontsize=18)

    plt.subplot(245)
    plt.grid(True)
    plt.plot(y2022_df['Hour'], y2022_df['Demand_MWh'], color='black', linestyle='-', label='Obs', linewidth=3)
    plt.plot(y2022_df['Hour'], y2022_df['MLP1_MWh'], color='rosybrown', linestyle='-', label=('MLP1 Corr=' + str(y2022_df['Demand_MWh'].corr(y2022_df['MLP1_MWh']).round(2))), linewidth=3)
    plt.plot(y2022_df['Hour'], y2022_df['MLP2_MWh'], color='firebrick', linestyle='-', label=('MLP2 Corr=' + str(y2022_df['Demand_MWh'].corr(y2022_df['MLP2_MWh']).round(2))), linewidth=3)
    plt.plot(y2022_df['Hour'], y2022_df['MLP3_MWh'], color='darkorange', linestyle='-', label=('MLP3 Corr=' + str(y2022_df['Demand_MWh'].corr(y2022_df['MLP3_MWh']).round(2))), linewidth=3)
    plt.plot(y2022_df['Hour'], y2022_df['MLP4_MWh'], color='olivedrab', linestyle='-', label=('MLP4 Corr=' + str(y2022_df['Demand_MWh'].corr(y2022_df['MLP4_MWh']).round(2))), linewidth=3)
    plt.plot(y2022_df['Hour'], y2022_df['MLP5_MWh'], color='deepskyblue', linestyle='-', label=('MLP5 Corr=' + str(y2022_df['Demand_MWh'].corr(y2022_df['MLP5_MWh']).round(2))), linewidth=3)
    plt.legend(loc='upper left', prop={'size': 18})
    plt.xlim([0, 23])
    plt.ylim([y_min, y_max])
    plt.xlabel('Hour of Day [UTC]', fontsize=18)
    plt.ylabel('Load [MWh]', fontsize=18)
    plt.title(('Load in ' + ba_to_plot + ' 2022: ' + season_to_plot), fontsize=21)
    plt.title('e)', loc='left', fontsize=18)

    plt.subplot(246)
    plt.grid(True)
    plt.plot(y2023_df['Hour'], y2023_df['Demand_MWh'], color='black', linestyle='-', label='Obs', linewidth=3)
    plt.plot(y2023_df['Hour'], y2023_df['MLP1_MWh'], color='rosybrown', linestyle='-', label=('MLP1 Corr=' + str(y2023_df['Demand_MWh'].corr(y2023_df['MLP1_MWh']).round(2))), linewidth=3)
    plt.plot(y2023_df['Hour'], y2023_df['MLP2_MWh'], color='firebrick', linestyle='-', label=('MLP2 Corr=' + str(y2023_df['Demand_MWh'].corr(y2023_df['MLP2_MWh']).round(2))), linewidth=3)
    plt.plot(y2023_df['Hour'], y2023_df['MLP3_MWh'], color='darkorange', linestyle='-', label=('MLP3 Corr=' + str(y2023_df['Demand_MWh'].corr(y2023_df['MLP3_MWh']).round(2))), linewidth=3)
    plt.plot(y2023_df['Hour'], y2023_df['MLP4_MWh'], color='olivedrab', linestyle='-', label=('MLP4 Corr=' + str(y2023_df['Demand_MWh'].corr(y2023_df['MLP4_MWh']).round(2))), linewidth=3)
    plt.plot(y2023_df['Hour'], y2023_df['MLP5_MWh'], color='deepskyblue', linestyle='-', label=('MLP5 Corr=' + str(y2023_df['Demand_MWh'].corr(y2023_df['MLP5_MWh']).round(2))), linewidth=3)
    plt.plot(y2023_df['Hour'], y2023_df['MLP6_MWh'], color='blueviolet', linestyle='-', label=('MLP6 Corr=' + str(y2023_df['Demand_MWh'].corr(y2023_df['MLP6_MWh']).round(2))), linewidth=3)
    plt.legend(loc='upper left', prop={'size': 18})
    plt.xlim([0, 23])
    plt.xlabel('Hour of Day [UTC]', fontsize=18)
    plt.ylabel('Load [MWh]', fontsize=18)
    plt.title(('Load in ' + ba_to_plot + ' 2023: ' + season_to_plot), fontsize=21)
    plt.title('f)', loc='left', fontsize=18)

    plt.subplot(247)
    plt.grid(True)
    plt.plot(y2024_df['Hour'], y2024_df['Demand_MWh'], color='black', linestyle='-', label='Obs', linewidth=3)
    plt.plot(y2024_df['Hour'], y2024_df['MLP1_MWh'], color='rosybrown', linestyle='-', label=('MLP1 Corr=' + str(y2024_df['Demand_MWh'].corr(y2024_df['MLP1_MWh']).round(2))), linewidth=3)
    plt.plot(y2024_df['Hour'], y2024_df['MLP2_MWh'], color='firebrick', linestyle='-', label=('MLP2 Corr=' + str(y2024_df['Demand_MWh'].corr(y2024_df['MLP2_MWh']).round(2))), linewidth=3)
    plt.plot(y2024_df['Hour'], y2024_df['MLP3_MWh'], color='darkorange', linestyle='-', label=('MLP3 Corr=' + str(y2024_df['Demand_MWh'].corr(y2024_df['MLP3_MWh']).round(2))), linewidth=3)
    plt.plot(y2024_df['Hour'], y2024_df['MLP4_MWh'], color='olivedrab', linestyle='-', label=('MLP4 Corr=' + str(y2024_df['Demand_MWh'].corr(y2024_df['MLP4_MWh']).round(2))), linewidth=3)
    plt.plot(y2024_df['Hour'], y2024_df['MLP5_MWh'], color='deepskyblue', linestyle='-', label=('MLP5 Corr=' + str(y2024_df['Demand_MWh'].corr(y2024_df['MLP5_MWh']).round(2))), linewidth=3)
    plt.plot(y2024_df['Hour'], y2024_df['MLP6_MWh'], color='blueviolet', linestyle='-', label=('MLP6 Corr=' + str(y2024_df['Demand_MWh'].corr(y2024_df['MLP6_MWh']).round(2))), linewidth=3)
    plt.plot(y2024_df['Hour'], y2024_df['MLP7_MWh'], color='navy', linestyle='-', label=('MLP7 Corr=' + str(y2024_df['Demand_MWh'].corr(y2024_df['MLP7_MWh']).round(2))), linewidth=3)
    plt.legend(loc='upper left', prop={'size': 18})
    plt.xlim([0, 23])
    plt.ylim([y_min, y_max])
    plt.xlabel('Hour of Day [UTC]', fontsize=18)
    plt.ylabel('Load [MWh]', fontsize=18)
    plt.title(('Load in ' + ba_to_plot + ' 2024: ' + season_to_plot), fontsize=21)
    plt.title('g)', loc='left', fontsize=18)

    # If the "save_images" flag is set to true then save the plot to a .png file:
    if save_images == True:
       plt.savefig(os.path.join(image_output_directory + 'BA_Diurnal_Errors_' + ba_to_plot + '_' + season_to_plot + '.png'), dpi=image_resolution, bbox_inches='tight')
       plt.close()


## Make the Plots


In [34]:
# Loop over the BAs and make the plot for each one:
for ba in bas:
    for season in ['DJF', 'JJA']:
        plot_ba_diurnal_errors(ba_to_plot = ba,
                               season_to_plot = season,
                               composite_input_directory = composite_input_directory,
                               image_output_directory = image_output_directory, 
                               image_resolution = 150, 
                               save_images = True)
